In [19]:
# import dependencies
import torch
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms

# For ploting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import sys
import pickle

# for bleu score
from nltk.translate import bleu_score
from nltk import word_tokenize

useGPU = torch.cuda.is_available()

In [2]:
# define useful utility function
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    plt.figure()
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def progress(count, total, suffix=''):
    """ Shows the progress of a given action 
    
    @params:
    - count : the current count of done operations
    - total : the total number of operation to do
    - suffix : a message printed after the progress bar
    """
    
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '#' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ... %s\r' % (bar, percents, '%', suffix))
    sys.stdout.flush()

In [3]:
# define our transformation function
centre_crop = transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# load the data with cocoAPI this time we dont need transform
cap = dset.CocoCaptions(root = '/home/raille/coco-features/coco-dataset/train2017',
                        annFile = '/home/raille/coco-features/coco-dataset/annotations/captions_train2017.json',
                        transform=centre_crop)

loading annotations into memory...
Done (t=2.12s)
creating index...
index created!


In [5]:
# load the raw and PCA features
raw_features = torch.load('../data/raw-features-scaled.pt')
PCA_features = torch.load('../data/PCA-features-scaled.pt')

## Perform kNN

In [6]:
# define function to perform kNN
def select_all_except_row(tensor, i):
    """
        return all the tensor except the i-th row
        @params:
            - tensor: 2D tensor on which to perform the operation
            - i: the row to drop
        @return:
            - tensor: 2D tensor with the i-th row droped
    """
    if i == 0:
        return tensor[1:]
    elif i == (len(tensor) - 1):
        return tensor[:-1]
    else:
        return torch.cat((tensor[0:i], tensor[i+1:]), 0)
    
def knn_bruteforce_gpu(tensor, k, with_progress=False):
    """
        Compute a knn list with element being tuples (values, indices) ordered,
        of the k nearest rows for each row of a tensor.
        @params:
            - tensor: 2D tensor on which to perform the algorithm
            - k: the number of nearest neighbor to keep
        @return:
            - knn_list: list of ordered tuples with the value, indices of the k nearest row for each row
    """
    knn_list = []
    for i in range(len(tensor)):
        vec = tensor[i]
        # mat = select_all_except_row(tensor, i)
        mat = tensor
        distances = torch.norm(mat - vec.expand(mat.size()), 2, 1)
        knn_list.append(torch.topk(distances, k, dim=0, largest=False))
        if with_progress:
            progress(i,len(tensor),'Computing k-nn')
    return knn_list

In [ ]:
%%time
knn_raw_features_list = knn_bruteforce_gpu(raw_features, 2)

CPU times: user 3min 33s, sys: 2min 24s, total: 5min 57s
Wall time: 5min 57s


In [16]:
knn_raw_features_list[118286]

(
   0.0000
  11.3638
 [torch.cuda.FloatTensor of size 2 (GPU 0)], 
  1.1829e+05
  6.5211e+04
 [torch.cuda.LongTensor of size 2 (GPU 0)])

In [15]:
%%time
knn_features_pca_list = knn_bruteforce_gpu(PCA_features, 2)

CPU times: user 2min 8s, sys: 1min 21s, total: 3min 30s
Wall time: 3min 30s


In [17]:
[x[1][1] for x in knn_raw_features_list]

[58235,
 11610,
 111459,
 4,
 3,
 66255,
 97647,
 114372,
 43724,
 84694,
 85631,
 32340,
 81328,
 83874,
 99014,
 100059,
 106077,
 108246,
 6836,
 38621,
 77828,
 16982,
 89424,
 111407,
 99,
 23625,
 106975,
 64191,
 98959,
 5322,
 98094,
 89157,
 35037,
 67103,
 77571,
 77523,
 27877,
 35,
 88811,
 42736,
 6302,
 30568,
 50017,
 18912,
 29649,
 10830,
 61177,
 14392,
 99106,
 39665,
 60104,
 55240,
 3635,
 99515,
 49911,
 21553,
 61159,
 70388,
 8123,
 33377,
 86852,
 86747,
 72498,
 70374,
 81654,
 70374,
 65841,
 116461,
 100800,
 81428,
 98018,
 13944,
 24259,
 28817,
 63505,
 39176,
 69979,
 69668,
 32397,
 21607,
 115048,
 114439,
 70358,
 4560,
 13552,
 44797,
 94938,
 33783,
 76485,
 8142,
 98885,
 16227,
 6405,
 103199,
 86805,
 45840,
 88842,
 63260,
 11709,
 24,
 45796,
 63748,
 70440,
 50024,
 48936,
 103227,
 105711,
 69710,
 52914,
 9974,
 105946,
 17143,
 36319,
 56681,
 22464,
 66346,
 97591,
 21183,
 35976,
 120,
 119,
 9061,
 33738,
 70676,
 106278,
 75500,
 102582

In [21]:
# create the match index files for the bleu score script
pickle.dump([x[1][1] for x in knn_raw_features_list], open("../data/match_index_knn_raw.pl", "wb"))
pickle.dump([x[1][1] for x in knn_features_pca_list], open("../data/match_index_knn_pca.pl", "wb"))

## Assess Result

In [ ]:
# display the 10 first match for the raw features
for i in range(0,10):
    matched_index = knn_raw_features_list[i][1][1]
    imshow(torchvision.utils.make_grid([cap[i][0], cap[matched_index][0]], nrow=2))

In [ ]:
# display the 10 first match for the pca features
for i in range(0,10):
    matched_index = knn_features_pca_list[i][1][1]
    imshow(torchvision.utils.make_grid([cap[i][0], cap[matched_index][0]], nrow=2))

In [ ]:
def compute_sim_percentage_arrays(array_1, array_2):
    """
        Compute the percentage of similarity between 2 1D IntTensor of same lentgh
    """
    return (array_1 == array_2).sum()/len(array_1)

In [ ]:
%%time
compute_sim_percentage_arrays(np.array([x[1][1] for x in knn_features_pca_list]), 
                              np.array([x[1][1] for x in knn_raw_features_list]))

In [ ]:
# implement bleu score
def compute_bleu_score(cap, matched_index_list):
    total_bleu_score = []
    weights = [0.25, 0.25, 0.25, 0.25]
    for i, matched_index in enumerate(matched_index_list):
        score = []
        for caption in cap[i][1]:
            candidate_tokens = word_tokenize(caption.replace('.',''))
            references_tokens = [word_tokenize(i.replace('.','')) for i in cap[matched_index][1]] 
            score.append(bleu_score.sentence_bleu(references_tokens, candidate_tokens, weights))
        mean = np.mean(score)
        total_bleu_score.append(mean)
    return np.mean(total_bleu_score)

In [ ]:
matched_index_list = [x[1][0] for x in knn_features_pca_list]
matched_index_list[0:10]

In [ ]:
matched_index_list = [x[1][0] for x in knn_features_pca_list]
bscore = compute_bleu_score(cap, matched_index_list)
print("PCA bleu score:", bscore)

In [ ]:
matched_index_list = [x[1][0] for x in knn_raw_features_list]
bscore = compute_bleu_score(cap, matched_index_list)
print("Raw bleu score:", bscore)